# Data labeling with LLMs, validating with humans
This notebook provides example [EDSL](https://github.com/expectedparrot/edsl) code for conducting a data labeling task with large language models and validating responses with humans.
The example below consists of the following steps, which can be conducted entirely in EDSL code or interactively at your [Coop account](https://www.expectedparrot.com):

* Construct questions about a dataset, using a placeholder in each question for the individual piece of data to be labeled (each answer is a "label" for a piece of data)
* Combine the questions in a survey to administer them together
* *Optionally* create AI agent personas to answer the questions (e.g., if there is relevant expertise or background for the task)
* Select language models to generate the answers (for the agents, or without referencing any AI personas)
* Run the survey with the data, agents and models to generate a formatted dataset of results
* Select questions and data that you want to validate with humans to create a subset of your survey (or leave it unchanged to run the entire survey with humans)
* Send a web-based version of the survey to human respondents
* Compare LLM and human answers, and iterate on the data labeling survey as needed!

Before running the code below please see instructions on [getting started](https://www.expectedparrot.com/en/latest/getting-started) using Expected Parrot tools for AI research.

## Construct questions about a dataset
We start by creating questions about a dataset, where each answer will provide a "label" for each piece of data. 
EDSL comes with many [common question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from a model (multiple choice, linear scale, matrix, etc.).

We use a "scenario" placeholder in each question text for data that we want to add to it.
This method allows us to efficiently readminister a question for each piece of data.
[Scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html) can be created from many types of data, including PNG, PDF, CSV, docs, lists, tables, videos, and other types.

We combine the questions in a [survey](https://docs.expectedparrot.com/en/latest/surveys.html) in order to administer them together, asynchronously by default, or else according to any [logic or rules](https://docs.expectedparrot.com/en/latest/surveys.html#survey-rules-logic) that we want to add (e.g., skip/stop rules).

In [1]:
from edsl import ScenarioList, QuestionList, QuestionNumerical, Survey

q1 = QuestionList(
    question_name = "characters",
    question_text = "Name all of the characters in this show: {{ scenario.show }}"
)

q2 = QuestionNumerical(
    question_name = "years",
    question_text = "Identify the year this show first aired: {{ scenario.show }}"
)

scenarios = ScenarioList.from_source("list", "show", ["The Simpsons", "South Park", "I Love Lucy"])

questions = q1.loop(scenarios) + q2.loop(scenarios)

survey = Survey(questions)

## Generate data "labels" using LLMs
EDSL allows us to [specify the models](https://docs.expectedparrot.com/en/latest/language_models.html) that we want to use to answer the questions, and optionally [design AI agent personas](https://docs.expectedparrot.com/en/latest/agents.html) for the models to reference in answering the questions.
This can be useful if you want to reference specific expertise that is relevant to the labeling task.

We administer the questions by adding the scenarios, agents and models to the survey and calling the `run()` method.
This generates a formatted dataset of `Results` that we can analyze with [built-in methods for working with results](https://docs.expectedparrot.com/en/latest/results.html).

In [2]:
from edsl import Agent, AgentList, Model, ModelList

agents = AgentList([
    Agent(traits = {"persona":"You watch a lot of TV."})
])

models = ModelList([
    Model("gemini-2.5-flash", service_name = "google"),
    Model("gpt-4o", service_name = "openai")
])

results = survey.by(scenarios).by(agents).by(models).run()

Results are accessible at your Coop account (see link above ^) and at your workspace. 
We can inspect a list of all the components of the results:

In [3]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,agent.persona
4,answer.characters_0
5,answer.characters_1
6,answer.characters_2
7,answer.years_0
8,answer.years_1
9,answer.years_2


Here we select components to display in a table:

In [4]:
results.select("model", "persona", "characters_0", "years_0", "characters_1", "years_1", "characters_2", "years_2")

,model.model,agent.persona,answer.characters_0,answer.years_0,answer.characters_1,answer.years_1,answer.characters_2,answer.years_2
0,gemini-2.5-flash,You watch a lot of TV.,"['Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Grandpa Simpson', 'Apu Nahasapeemapetilon', 'Ned Flanders', 'Moe Szyslak', 'Barney Gumble', 'Chief Wiggum', 'Krusty the Clown', 'Milhouse Van Houten', 'Nelson Muntz', 'Lenny Leonard', 'Carl Carlson', 'Smithers', 'Burns', 'Sideshow Bob', 'Ralph Wiggum']",1989,"['Stan Marsh', 'Kyle Broflovski', 'Eric Cartman', 'Kenny McCormick', 'Randy Marsh', 'Sharon Marsh', 'Gerald Broflovski', 'Sheila Broflovski', 'Butters Stotch', 'Chef', 'Mr. Garrison', 'Mr. Mackey', 'Jimbo Kern', 'Ned Gerblansky', 'Officer Barbrady', 'Liane Cartman', 'Scott Tenorman', 'Wendy Testaburger', 'Heidi Turner', 'Token Black', 'Clyde Donovan', 'Craig Tucker', 'Tweek Tweak', 'Timmy Burch', 'Kevin Stoley', 'Ike Broflovski', 'Leopold Stotch']",1997,"['Lucy Ricardo', 'Ricky Ricardo', 'Fred Mertz', 'Ethel Mertz', 'Little Ricky Ricardo', 'Mr. and Mrs. Howard']",1951
1,gpt-4o,You watch a lot of TV.,"['Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abe Simpson', 'Ned Flanders', 'Milhouse Van Houten', 'Mr. Burns', 'Waylon Smithers', 'Krusty the Clown', 'Principal Skinner', 'Moe Szyslak', 'Barney Gumble', 'Chief Wiggum', 'Ralph Wiggum', 'Apu Nahasapeemapetilon', 'Sideshow Bob', 'Edna Krabappel', 'Patty Bouvier', 'Selma Bouvier', 'Comic Book Guy', 'Nelson Muntz', 'Groundskeeper Willie', 'Lenny Leonard', 'Carl Carlson', 'Dr. Hibbert', 'Reverend Lovejoy', 'Mayor Quimby', 'Martin Prince']",1989,"['Eric Cartman', 'Stan Marsh', 'Kyle Broflovski', 'Kenny McCormick', 'Randy Marsh', 'Butters Stotch', 'Mr. Garrison', 'Chef', 'Wendy Testaburger', 'Mr. Mackey']",1997,"['Lucy Ricardo', 'Ricky Ricardo', 'Ethel Mertz', 'Fred Mertz']",1951
2,gemini-2.5-flash,You watch a lot of TV.,"['Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abe Simpson', 'Clancy Wiggum', 'Ned Flanders', 'Moe Szyslak', 'Apu Nahasapeemapetilon', 'Krusty the Clown', 'Milhouse Van Houten', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson']",1989,"['Stan Marsh', 'Kyle Broflovski', 'Eric Cartman', 'Kenny McCormick', 'Randy Marsh', 'Sharon Marsh', 'Gerald Broflovski', 'Sheila Broflovski', 'Butters Stotch', 'Chef', 'Mr. Garrison', 'Mr. Mackey', 'Jimbo Kern', 'Ned Gerblansky', 'Officer Barbrady', 'Liane Cartman', 'Scott Tenorman', 'Wendy Testaburger', 'Heidi Turner', 'Token Black', 'Clyde Donovan', 'Craig Tucker', 'Tweek Tweak', 'Timmy Burch', 'Kevin Stoley', 'Ike Broflovski', 'Leopold Stotch']",1997,"['Lucy Ricardo', 'Ricky Ricardo', 'Fred Mertz', 'Ethel Mertz', 'Little Ricky Ricardo', 'Mr. Mooney']",1951
3,gpt-4o,You watch a lot of TV.,"['Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abe Simpson', 'Ned Flanders', 'Mr. Burns', 'Waylon Smithers', 'Moe Szyslak', 'Barney Gumble', 'Seymour Skinner', 'Edna Krabappel', 'Nelson Muntz', 'Milhouse Van Houten', 'Ralph Wiggum', 'Chief Wiggum', 'Apu Nahasapeemapetilon', 'Krusty the Clown', 'Sideshow Bob']",1989,"['Eric Cartman', 'Stan Marsh', 'Kyle Broflovski', 'Kenny McCormick', 'Butters Stotch', 'Randy Marsh', 'Mr. Garrison', 'Chef', 'Mr. Mackey', 'Wendy Testaburger', 'Jimmy Valmer', 'Timmy Burch', 'Terrance', 'Phillip', 'Token Black', 'Tweek Tweak', 'Craig Tucker', 'Clyde Donovan', 'Bebe Stevens', 'Ike Broflovski', 'Shelley Marsh', 'Gerald Broflovski', 'Sheila Broflovski', 'Sharon Marsh', 'Liane Cartman', 'Principal Victoria', 'Officer Barbrady']",1997,"['Lucy Ricardo', 'Ricky Ricardo', 'Ethel Mertz', 'Fred Mertz']",1951
4,gemini-2.5-flash,You watch a lot of TV.,"['Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Grandpa Simpson', 'Apu Nahasapeemapetilon', 'Ned Flanders', 'Moe Szyslak', 'Barney Gumble', 'Chief Wiggum', 'Krusty the Clown', 'Milhouse Van Houten', 'Nelson Muntz', 'Princi

## Run the survey with human respondents
We can validate some of all of the responses with human respondents by calling the `humanize()` method on the version of the survey that we want to validate with humans.
This method generates a shareable URL for a web-based version of the survey that you can distribute, together with a URL for tracking the responses at your Coop account.

Here we create a new version of the survey to add some screening/information questions of the humans that answer it:

In [5]:
from edsl import QuestionLinearScale

q3 = QuestionLinearScale(
    question_name = "tv_viewing",
    question_text = "On a scale from 1 to 5, how much tv would you say that you've watched in your life?",
    question_options = [1,2,3,4,5],
    option_labels = {
        1:"None at all",
        5:"A ton"
    }
)

q4 = QuestionNumerical(
    question_name = "age",
    question_text = "How old are you (in years)?"
)

new_questions = [q3, q4]

human_survey = Survey(questions + new_questions)

In [6]:
human_survey.humanize()

{'project_name': 'Project',
 'uuid': 'bbb84776-3364-4bc9-b028-0119cd84d480',
 'admin_url': 'https://www.expectedparrot.com/home/projects/bbb84776-3364-4bc9-b028-0119cd84d480',
 'respondent_url': 'https://www.expectedparrot.com/respond/bbb84776-3364-4bc9-b028-0119cd84d480'}

Responses automatically appear at your Coop account, and you can import them into your workspace using `Coop` methods:

In [7]:
from edsl import Coop

human_results = Coop().get_project_human_responses("bbb84776-3364-4bc9-b028-0119cd84d480")
human_results

,characters_1,age,years_1,tv_viewing,years_0,characters_0,years_2,characters_2,scenario_index,agent_index,agent_instruction,agent_name,model,temperature,inference_service,model_index,characters_2_user_prompt,characters_2_system_prompt,years_1_system_prompt,years_2_user_prompt,years_2_system_prompt,years_1_user_prompt,characters_0_user_prompt,years_0_user_prompt,characters_1_system_prompt,characters_0_system_prompt,years_0_system_prompt,characters_1_user_prompt,tv_viewing_user_prompt,tv_viewing_system_prompt,age_user_prompt,age_system_prompt,characters_2_input_tokens,years_1_one_usd_buys,age_output_price_per_million_tokens,tv_viewing_cost,years_1_input_tokens,years_1_output_price_per_million_tokens,characters_1_output_tokens,years_2_one_usd_buys,years_1_cost,characters_2_output_price_per_million_tokens,characters_0_one_usd_buys,years_2_input_price_per_million_tokens,years_1_raw_model_response,years_0_input_tokens,characters_1_output_price_per_million_tokens,characters_0_input_tokens,years_2_raw_model_response,tv_viewing_one_usd_buys,characters_1_input_price_per_million_tokens,characters_2_input_price_per_million_tokens,age_input_tokens,years_2_input_tokens,tv_viewing_raw_model_response,years_2_output_tokens,characters_2_raw_model_response,age_input_price_per_million_tokens,years_0_one_usd_buys,characters_1_raw_model_response,years_2_output_price_per_million_tokens,years_0_output_tokens,tv_viewing_input_tokens,years_0_cost,age_cost,characters_2_output_tokens,characters_1_input_tokens,age_one_usd_buys,tv_viewing_output_tokens,characters_2_cost,characters_0_output_price_per_million_tokens,characters_0_output_tokens,years_1_input_price_per_million_tokens,years_1_output_tokens,characters_0_input_price_per_million_tokens,age_output_tokens,characters_1_cost,years_0_input_price_per_million_tokens,characters_1_one_usd_buys,characters_0_cost,characters_0_raw_model_response,years_0_raw_model_response,tv_viewing_output_price_per_million_tokens,age_raw_model_response,years_2_cost,years_0_output_price_per_million_tokens,characters_2_one_usd_buys,tv_viewing_input_price_per_million_tokens,iteration,characters_1_question_text,characters_2_question_text,tv_viewing_question_text,years_1_question_text,years_2_question_text,characters_0_question_text,years_0_question_text,age_question_text,characters_0_question_options,years_1_question_options,characters_2_question_options,tv_viewing_question_options,characters_1_question_options,years_0_question_options,years_2_question_options,age_question_options,characters_1_question_type,characters_2_question_type,age_question_type,tv_viewing_question_type,characters_0_question_type,years_1_question_type,years_2_question_type,years_0_question_type,age_comment,tv_viewing_comment,characters_2_comment,characters_0_comment,years_0_comment,characters_1_comment,years_1_comment,years_2_comment,characters_1_generated_tokens,years_0_generated_tokens,tv_viewing_generated_tokens,characters_2_generated_tokens,characters_0_generated_tokens,years_1_generated_tokens,age_generated_tokens,years_2_generated_tokens,characters_2_cache_used,tv_viewing_cache_used,characters_1_cache_used,years_0_cache_used,years_1_cache_used,characters_0_cache_used,age_cache_used,years_2_cache_used,characters_2_cache_key,years_0_cache_key,age_cache_key,years_2_cache_key,characters_0_cache_key,characters_1_cache_key,years_1_cache_key,tv_viewing_cache_key,characters_2_reasoning_summary,tv_viewing_reasoning_summary,years_1_reasoning_summary,years_2_reasoning_summary,years_0_reasoning_summary,characters_1_reasoning_summary,characters_0_reasoning_summary,age_reasoning_summary
0,"['Cartman', 'Stewie']",46,1998,5,1989,"['Homer', 'Marge', 'Randall', 'Bort']",1953,"['Lucy ', 'Dezi']",0,0,nan,a782895b-e5dc-41cb-80e8-8a956db1ee18,test,0.500000,test,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,Not Applicable,nan,nan,nan,Not Applicable,nan,nan,nan,nan,nan,Not Applicable,nan,Not Applicable,nan,n

In [8]:
human_results.select("age", "tv_viewing", "characters_0", "years_0", "characters_1", "years_1", "characters_2", "years_2")

,answer.age,answer.tv_viewing,answer.characters_0,answer.years_0,answer.characters_1,answer.years_1,answer.characters_2,answer.years_2
0,46,5,"['Homer', 'Marge', 'Randall', 'Bort']",1989,"['Cartman', 'Stewie']",1998,"['Lucy ', 'Dezi']",1953
1,11,4,"['Homer', 'Marge', 'Bart', 'Lisa', 'Maggie', 'Grandpa', 'Mr. Burns', 'Apu', 'Snake', 'Moe', 'Krusty', 'Lenny', 'Carl', 'Barney', 'Smithers', 'Sideshow Mel', 'Patty', 'Selma', 'Martin', 'Nelson', 'Ralph', 'Chief Wiggum', 'Reverend Lovejoy', ""Santa's Little Helper"", 'Snowball II', 'Miss Crabapple', 'Miss Hoover', 'Principal Skinner', 'Willie', 'Superintendent Chalmers', 'Lou', 'Comic Book Guy', 'Sherry', 'Terry', 'Fat Tony', 'Johnny Tightlips', 'Jimmy the Squealer', 'Mayor Quimby', 'Sideshow Bob', 'Luigi', 'Spiderpig', 'Duffman', 'Larry', 'Grandma', 'Mr. Teasy']",1989,"[""I don't know""]",2000,['Uhhhhh no'],2010
